
# <center>时尚物品识别案例</center>

## 使用CNN时尚物品识别
  * 彩色图片n✖m✖3

CIFAR-10由10类中的60000张32x32彩色图像组成，每类有6000张32\*32的彩色图像。有50000张训练图像和10000张测试图像。有10类，飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车，每类6000张图。与MNIST相比，色彩、颜色噪点较多，同一类物体大小不一、角度不同、颜色不同。
以下是数据集中的类，以及每个中的10个随机图像：

![](images/CIFAR-10.png)

#### 加载数据

In [ ]:
# 导入第三方包
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

首先载入cifar10数据集，和mnist数据集的载入方法一致，本地没有数据的话会先下载

In [ ]:
# 加载数据集
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

cifar10数据集图像大小是32*32的3通道彩图，训练集5万张，测试集1万张。和之前的mnist数据集不同，由于是彩色的，所以样本直接就是4维的。

In [ ]:
# 查看数据形状
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

#### 预览数据

In [ ]:
# 可视化部分数据
import matplotlib.pyplot as plt

label_dict={0:"airplane",1:"automobile",2:"bird",3:"cat",4:"deer",
            5:"dog",6:"frog",7:"horse",8:"ship",9:"truck"}
plt.figure(figsize=(15, 6))
for i in range(0, 10):
    plt.subplot(2, 5, i+1)
    x_selected = x_train[i]
    plt.imshow(x_train[i])
    plt.title(str(i)+','+label_dict[y_train[i][0]])
    plt.axis('off')
plt.show()

#### 规范化数据

In [ ]:
# 查看数据
x_train[0]

In [ ]:
# 对数据X归一化
x_train = x_train/255
x_test = x_test/255

In [ ]:
# 对数据y热编码
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)

#### 构建模型

In [ ]:
# 构造LeNet网络
model = Sequential()

model.add(Conv2D(filters=6, kernel_size=(3, 3),input_shape=(32,32,3), activation='tanh'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=16, kernel_size=(3, 3),activation='tanh'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(120, activation='tanh'))

model.add(Dense(84, activation='tanh'))

model.add(Dense(10, activation='softmax'))

In [ ]:
# 打印网络模型结构
model.summary()

In [ ]:
# 编译模型
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics=['accuracy'])

In [ ]:
# 训练模型
train_history = model.fit(x_train,y_train,batch_size=64,epochs=5,validation_data=(x_test,y_test))

In [ ]:
# 可视化训练过程
import matplotlib.pyplot as plt
plt.plot(train_history.history['accuracy'])
plt.plot(train_history.history['val_accuracy'])
plt.title('Train History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# 评估模型
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#### 预测结果

In [ ]:
# 预测数据
prediction=model.predict_classes(x_test)
prediction[:10]

In [ ]:
# 可视化预测结果
import matplotlib.pyplot as plt

label_dict={0:"airplane",1:"automobile",2:"bird",3:"cat",4:"deer",
            5:"dog",6:"frog",7:"horse",8:"ship",9:"truck"}

def plot_images_labels_prediction(images,labels,prediction,idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12, 14)
    if num>25: num=25 
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[idx],cmap='binary')
                        
        title=str(i)+','+label_dict[labels[i][0]]
        if len(prediction)>0:
            title+='=>'+label_dict[prediction[i]]
            
        ax.set_title(title,fontsize=10) 
        ax.set_xticks([]);ax.set_yticks([])        
        idx+=1 
    plt.show()

plot_images_labels_prediction(x_test,y_test,prediction,0,10)

#### 显示混淆矩阵

In [ ]:
# 以混淆矩阵展现结果，对角线为预测正确的
import pandas as pd
(x_Train, y_Train), (x_Test, y_Test) = cifar10.load_data()
pd.crosstab(y_Test.reshape(-1),prediction,
            rownames=['label'],colnames=['predict'])

```
0:"airplane",1:"automobile",2:"bird",3:"cat",4:"deer", 5:"dog",6:"frog",7:"horse",8:"ship",9:"truck"
```

* 对角线
  * ship 预测最准确，778/1000，说明最不容易被混淆
  * cat 预测最不准，410/1000，说明最容易被混淆

* 非对角线
  * 把 cat 预测成 dog 最多，169
  * 把 dog 预测成 cat 第三多，149 ，猫狗很容易混淆

#### 数据增强

为了模型更快的收敛以及更好的泛化性能，往往我们会对图像做一些变换，比如缩放、平移、旋转等等。

下面我们要用keras自带的图像增强来对图像做一些变换。

这里生成了一个数据增强器，包含了范围20°内的随机旋转，±15%的缩放以及随机的水平翻转。可调的参数还有很多，具体的可以查看文档。
![](images/idg.png)

In [ ]:
# 构造数据增强器
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range = 20,
    zoom_range = 0.15,
    horizontal_flip = True,
)

In [ ]:
# 增强训练数据X
datagen.fit(x_train)

In [ ]:
# 训练模型
model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                        steps_per_epoch=750,
                        epochs=5,
                        validation_data=(x_test,y_test), workers=4)

通过ImageDataGenerator生成的数据需要使用model的fit_generator方法来进行训练，其中的workers参数表示多线程运算。

datagen的flow方法可以按批次的生成训练所需数据，注意这里生成的数据都是经过了数据增强的，并且是实时的。

In [ ]:
# 评估模型
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### 总结

1. 学习了使用Keras内置的ImageDataGenerator来做数据增强的方法
1. 调用model的fit_generator来进行针对增强数据的训练

# Any Questions?